In [8]:
import pickle 
import numpy as np
import pandas as pd
import statistics
from PIL import Image
import os 
import random
import time
import matplotlib.pyplot as plt
import netaddr

In [9]:
import dgl
import torch
import torch.nn as nn
import dgl.function as fn
import dgl.nn as dglnn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [10]:
#pip install netaddr

In [11]:
#directory should change based on the device, for laptop it should be users//jalal for office desktop it is users//jghadermazi

In [12]:
wed_data = pd.read_pickle(r'C:\\Users\\Jalal\\Box\\GAN Project\\Sample data\\Thursday_Packet_Level_Data.pkl')

In [13]:
df1 = wed_data.copy()

In [14]:
del wed_data

In [15]:
df1[['Payload']] = df1[['Payload']].replace('', '00')

In [16]:
df1

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,1499343600.000989000,192.168.10.16,209.58.128.160,39846,443,3529,897,00c1b114eb310023ae9b8abf0800,4500003445b240004006d87ec0a80a10d13a80a0,9ba601bb201715c34e47065b8010015cc3a60000010108...,00,fwd,1.0,8.0
1,1499343600.000991000,192.168.10.16,23.60.139.27,46500,80,3537,432,00c1b114eb310023ae9b8abf0800,45000034574e400040067666c0a80a10173c8b1b,b5a40050562f11268d8528b680100112f4ee0000010108...,00,bwd,1.0,8.0
2,1499343600.001028000,192.168.10.16,23.60.139.27,46498,80,3536,432,00c1b114eb310023ae9b8abf0800,45000034cee640004006fecdc0a80a10173c8b1b,b5a200500271988f98a44e71801001008b460000010108...,00,fwd,1.0,8.0
3,1499343600.001547000,216.58.219.226,192.168.10.25,443,49221,3561,3500,002500a8c46000c1b114eb310800,450005b92a8100003506d6dfd83adbe2c0a80a19,01bbc045d127a8cca9a506308018015d92590000010108...,170301058065e6491e2fc3dc9d004ef8bc528b237ab74f...,fwd,1.0,8.0
4,1499343600.001577000,192.168.10.25,216.58.219.226,49221,443,3561,806,00c1b114eb31002500a8c4600800,450000345e9f400040065d46c0a80a19d83adbe2,c04501bba9a50630d127ae518010ffcfc92d0000010108...,00,bwd,1.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7187244,1499365265.817911000,206.126.112.142,192.168.10.16,443,42198,99781,4909563,0023ae9b8abf00c1b114eb310800,450005dcf0ae0000390681a8ce7e708ec0a80a10,01bba4d6f40edf7af598c863801000b7b3290000010108...,59210587b21ed3a8d3c3c35f16fa39c781507dfc743cfc...,bwd,0.0,0.0
7187245,1499365265.818102000,206.126.112.142,192.168.10.16,443,42198,99781,4911011,0023ae9b8abf00c1b114eb310800,450005dcf0af0000390681a7ce7e708ec0a80a10,01bba4d6f40ee522f598c863801000b7168a0000010108...,c92242f4b3a7d9a4c9cc0a701d995a7ac4c682158dadd2...,bwd,0.0,0.0
7187246,1499365265.818295000,206.126.112.142,192.168.10.16,443,42198,99781,4912459,0023ae9b8abf00c1b114eb310800,45000b84f0b0000039067bfece7e708ec0a80a10,01bba4d6f40eeacaf598c863801000b7153c0000010108...,988822bedfd4cf1c3958c978619bf4fa79fcdd10d45749...,bwd,0.0,0.0
7187247,1499365265.818487000,206.126.112.142,192.168.10.16,443,42198,99781,4915355,0023ae9b8abf00c1b114eb310800,45000b84f0b2000039067bfcce7e708ec0a80a10,01bba4d6f40ef61af598c863801000b7153c0000010108...,7c8dcb764403b28792b82bce3b6e9fb9a86771a3093069...,bwd,0.0,0.0


In [17]:
#benign    0
#dos slowloris    1
#dos slowhttptest   2
#dos hulk     3
#dos goldeneye    4
#heartbleed    5
#ftp patator    6
#ssh patator    7
#webattack bruteforce    8
#webattack xss     9
#webattack sql injection 10
#infiltration     11
#botnet     12
#portscan   13
#ddos     14

In [18]:
df1['Class Label'].value_counts()

0.0     7087672
11.0      59754
8.0       30059
9.0        9638
10.0        126
Name: Class Label, dtype: int64

In [19]:
df = df1[df1['Class Label']== 11]

In [20]:
df = df.reset_index(inplace = False)

In [21]:
df

,index,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,39823,1499352852.592443000,192.168.10.25,192.168.10.3,50889,389,34376,1345,1866da9be37d002500a8c4600800,4500004ab65540004006eeebc0a80a19c0a80a03,c6c90185825a76a62429dc278018ffffff7a0000010108...,3014020103600f0201030400a3080406475353415049,fwd,1.0,11.0
1,39824,1499352852.592507000,192.168.10.3,192.168.10.25,389,50889,34376,3093,002500a8c4601866da9be37d0800,4500006c6507400080060018c0a80a03c0a80a19,0185c6c92429dc27825a76bc8018081cf6d80000010108...,3084000000320201036184000000290a010e0400040087...,bwd,1.0,11.0
2,39825,1499352852.592511000,192.168.10.3,192.168.10.25,389,50889,34376,3093,002500a8c4601866da9be37d0800,4500006c6507400080060018c0a80a03c0a80a19,0185c6c92429dc27825a76bc8018081cf6d80000010108...,3084000000320201036184000000290a010e0400040087...,fwd,1.0,11.0
3,39826,1499352852.592560000,192.168.10.25,192.168.10.3,50889,389,34376,1367,1866da9be37d002500a8c4600800,45000034c60540004006df51c0a80a19c0a80a03,c6c90185825a76bc2429dc5f8010ffffdaaf0000010108...,00,bwd,1.0,11.0
4,39827,1499352852.592716000,192.168.10.25,192.168.10.3,50889,389,34376,1367,1866da9be37d002500a8c4600800,4500006c0f1f400040069600c0a80a19c0a80a03,c6c90185825a76bc2429dc5f8018ffff978f0000010108...,303602010460310201030400a32a040647535341504904...,fwd,1.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59749,99572,1499363229.477506000,199.167.65.25,192.168.10.17,443,54467,64564,7550,0023ae9b956700c1b114eb310800,450005dcabbe40003706bee3c7a74119c0a80a11,01bbd4c3aaf4077bc3e293898010007f64a20000010108...,9e300b66c5d0a70cb363bbffd8b4e463596a1fe1b4fd4a...,bwd,1.0,11.0
59750,99573,1499363269.857641000,104.27.33.64,192.168.10.15,80,58962,64788,56464,001e4fd4ca2800c1b114eb310800,450005dc8a7d400039065d8c681b2140c0a80a0f,0050e6520209e45bb51caad4501000202a300000,3b9172003e7c6645899bf28f81ab482d0b9b9709ecc578...,bwd,1.0,11.0
59751,99574,1499363269.857873000,104.27.33.64,192.168.10.15,80,58962,64788,57924,001e4fd4ca2800c1b114eb310800,450005dc0a7e40003906dd8b681b2140c0a80a0f,0050e6520209ea0fb51caad45010002053630000,cf40c0576f3d87e302a0a14852866b280f3a6f269e407d...,fwd,1.0,11.0
59752,99575,1499363296.580451000,192.168.10.15,52.94.234.174,59016,80,64903,275,00c1b114eb31001e4fd4ca280800,4500002917c240008006f948c0a80a0f345eeaae,e6880050a7ba2e34eb526fe2501000ffad130000,000000000000,fwd,1.0,11.0


In [22]:
ip_header_lst = []

for i in range(len(df)):
    x = df['IP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 9, 12, 13, 14, 15, 16, 17, 18, 19])
    
    output = np.abs(output.astype(float))/255
    ip_header_lst.append(output)
    
df['IP Header Array'] = ip_header_lst

In [23]:
tcp_header_lst = []

for i in range(len(df)):
    x = df['TCP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 2, 3])
    output = np.abs(output.astype(float))/255
    
    tcp_header_lst.append(output)
    
df['TCP Header Array'] = tcp_header_lst

In [24]:
df['pkt_header_values'] = df.apply(lambda x : (np.append(x['IP Header Array'], x['TCP Header Array'])), axis=1)

In [25]:
new_cols = ['Source IP', 'Dst IP', 'Source Port', 'Dst Port','Epoch Time','Stream Index', 'Sequence Number', 'Direction', 'pkt_header_values']
df = df[new_cols]

In [26]:
# 70% of benign data for train, 30% for test benign

In [27]:
#random.seed(0)
#mlist = random.sample(streams_lst, 61492)

In [28]:
#len(imlist)

In [29]:
#df_train = df.loc[df['Stream Index'].isin(imlist)]
#df_train = df_train.reset_index(inplace = False)

In [30]:
#df_test = df.loc[~df['Stream Index'].isin(imlist)]
#df_test = df_test.reset_index(inplace = False)

In [31]:
#streams_lst1 = list(df_test['Stream Index'].unique())

In [32]:
#len(streams_lst1)

In [33]:
df

,Source IP,Dst IP,Source Port,Dst Port,Epoch Time,Stream Index,Sequence Number,Direction,pkt_header_values
0,192.168.10.25,192.168.10.3,50889,389,1499352852.592443000,34376,1345,fwd,"[0.0, 0.2901960784313726, 0.7137254901960784, ..."
1,192.168.10.3,192.168.10.25,389,50889,1499352852.592507000,34376,3093,bwd,"[0.0, 0.4235294117647059, 0.396078431372549, 0..."
2,192.168.10.3,192.168.10.25,389,50889,1499352852.592511000,34376,3093,fwd,"[0.0, 0.4235294117647059, 0.396078431372549, 0..."
3,192.168.10.25,192.168.10.3,50889,389,1499352852.592560000,34376,1367,bwd,"[0.0, 0.20392156862745098, 0.7764705882352941,..."
4,192.168.10.25,192.168.10.3,50889,389,1499352852.592716000,34376,1367,fwd,"[0.0, 0.4235294117647059, 0.058823529411764705..."
...,...,...,...,...,...,...,...,...,...
59749,199.167.65.25,192.168.10.17,443,54467,1499363229.477506000,64564,7550,bwd,"[0.0196078431372549, 0.8627450980392157, 0.670..."
59750,104.27.33.64,192.168.10.15,80,58962,1499363269.857641000,64788,56464,bwd,"[0.0196078431372549, 0.8627450980392157, 0.541..."
59751,104.27.33.64,192.168.10.15,80,58962,1499363269.857873000,64788,57924,fwd,"[0.0196078431372549, 0.8627450980392157, 0.039..."
59752,192.168.10.15,52.94.234.174,59016,80,1499363296.580451000,64903,275,fwd,"[0.0, 0.1607843137254902, 0.09019607843137255,..."


In [34]:
streams_lst = list(df['Stream Index'].unique())
print('Total number of flows in class:', len(streams_lst))    
random.shuffle(streams_lst)

# Directory path where to save the CSV files
directory_path = 'Thursday_infiltration_all/'

for PKT in range(2, 16):
    
    file_name = f'thursday_infiltration_{PKT}pkt.csv'
    file_path = directory_path + file_name
    

    data = []
    num_pkt = PKT
    pkt_val_pad = [0] * 25


    for j in range(len(streams_lst)):

        df_stream = df[df['Stream Index'] == streams_lst[j]]


        df_stream = df_stream.sort_values(by = 'Epoch Time')
        df_stream = df_stream.reset_index(drop=True)


        if len(df_stream) >= num_pkt:

            src_node = (int(netaddr.IPAddress(df_stream['Source IP'][0])), int(df_stream['Source Port'][0]))
            dst_node = (int(netaddr.IPAddress(df_stream['Dst IP'][0])), int(df_stream['Dst Port'][0]))
            stream_no = df_stream['Stream Index'][0]

            pkt_val_seq = []

            for k in range(0, num_pkt):

                pkt_val = []
                if df_stream['Direction'][k] == 'fwd':
                    pkt_val = [*df_stream['pkt_header_values'][k], *pkt_val_pad]
                else:
                    pkt_val = [*pkt_val_pad, *df_stream['pkt_header_values'][k]]

                pkt_val_seq.extend(pkt_val)

            data.append([src_node, dst_node, stream_no, pkt_val_seq])

    df2 = pd.DataFrame(data, columns= ['src_node', 'dst_node', 'stream_number', 'pkt_vals'])
    
    df2.to_csv(file_path, index=False)

Total number of flows in class: 4254
